In [ ]:
%load_ext autoreload
%autoreload 2

import hydra
import os
import datetime
from pathlib import Path

# Initialize hydra and move to the root of the repository
try:
    hydra.initialize(version_base=None, config_path="../config/")
    CONFIG = hydra.compose(config_name="main.yaml")
    print('Initializing hydra')
except:
    print('Hydra already initalized!')
else:
    os.chdir('..')

# Create an output folder in the root of the repository
OUTPUT_FOLDER = Path('output/{0}'.format(datetime.datetime.now()))
Path(OUTPUT_FOLDER).mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np

In [ ]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [ ]:
nodes = pd.read_csv('data/01_raw/city_nodes_annual_all.csv')
edges = pd.read_csv('data/01_raw/city_edges_annual_all.csv')

pos = nodes[['PlaceID', 'XCOORD', 'YCOORD']].drop_duplicates().set_index('PlaceID').to_dict('index')
pos = {key: (value['XCOORD'] - 3000000, value['YCOORD']) for key, value in pos.items()}

years = np.sort(edges.Year.unique())

In [ ]:
def weighted_network():
    edge_weights = edges.groupby(['from', 'to']).Year.count().reset_index().rename(columns={'Year': 'weight'})
    edge_weights.weight /= edge_weights.weight.max()
    edge_weights = edge_weights.values.tolist()
    edge_weights = [[int(e[0]), int(e[1]), e[2]] for e in edge_weights]
    
    G = nx.Graph()
    G.add_weighted_edges_from(edge_weights)

    return G

def singular_network(year, edges):
    edges = edges[edges.Year == year]
    edges = edges[['from', 'to']].values.tolist()
    return edges

In [ ]:
n_edges = []
for year in years:
    e = singular_network(year, edges)
    n_edges.append(len(e))

In [ ]:
f, ax0 = plt.subplots()

nodes.groupby('Year').Juden.sum().plot(ax=ax0)

ax1 = ax0.twinx()
ax1.plot(years, n_edges, c='k')

In [ ]:
comms = nx.community.louvain_communities(G)
T = np.sort([len(c) for c in comms])[-10]
comms = [c for c in comms if len(c) >= T]

print(len(comms))
assert len(comms) <= 10

In [ ]:
nx.draw_networkx_nodes(G, pos, node_size=12, node_color='k')

for i, comm in enumerate(comms):
    nx.draw_networkx_nodes(G, pos, nodelist=comm, node_size=15, node_color=colors[i])

nx.draw_networkx_edges(G, pos, alpha=0.025)